<a href="https://colab.research.google.com/github/Julian-Ojeda/very-simple-Multilayer-Perceptron-MLP-only-numpy-/blob/main/perceptron_multicapa_a_pelo_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

# --- DATOS: XOR clásico ---
x_xor = np.array([        # x ∈ ℝ^{4 × 2}
    [-1,  1],
    [ 1, -1],
    [-1, -1],
    [ 1,  1]
])

y_xor = np.array([        # y ∈ ℝ^{4 × 1} (codificación bipolar {-1, 1})
    [ 1],
    [ 1],
    [-1],
    [-1]
])

# --- FUNCIONES AUXILIARES ---

def agregar_sesgo(x):

    p = x.shape[0]
    sesgo = np.ones((p, 1))         # sesgo ∈ ℝ^{p × 1}
    return np.hstack((x, sesgo))    # concatena horizontalmente

def g(h, β=1):

    return np.tanh(β * h)

def g_derivada(h, β=1):

    return β * (1 - g(h, β)**2)

# --- ENTRENAMIENTO DEL PERCEPTRÓN MULTICAPA ---

def entrenar_perceptron(x, y, η=0.05, COTA=100000):
    np.random.seed(42)

    # y ∈ ℝ^{p × 1}, se aplica g() solo por coherencia de escala
    y = g(y)

    # Agregamos sesgo: x ∈ ℝ^{p × 2} → x̃ ∈ ℝ^{p × 3}
    x = agregar_sesgo(x)

    # Inicialización de pesos:
    # wjk ∈ ℝ^{3 × 3}  → pesos desde capa de entrada (3 neuronas con sesgo) a capa oculta (3 neuronas)
    # wij ∈ ℝ^{3 × 1}  → pesos desde capa oculta (3 neuronas) a salida (1 neurona)
    wjk = np.random.uniform(-1, 1, (3, 3))
    wij = np.random.uniform(-1, 1, (3, 1))

    i = 0
    error = 50000
    error_min = 1

    while error > 0 and i < COTA:
        indice = np.random.randint(0, x.shape[0])

        Vk = x[indice].reshape(1, -1)   # Vk ∈ ℝ^{1 × 3}

        # PROPAGACIÓN HACIA ADELANTE

        h_j = np.dot(Vk, wjk)           # h_j ∈ ℝ^{1 × 3} = (1×3)·(3×3)
        Vj = g(h_j)                     # Vj ∈ ℝ^{1 × 3}

        h_i = np.dot(Vj, wij)          # h_i ∈ ℝ^{1 × 1} = (1×3)·(3×1)
        Vi = g(h_i)                    # Vi ∈ ℝ^{1 × 1}

        # RETROPROPAGACIÓN

        δi = g_derivada(h_i) * (y[indice] - Vi)     # δi ∈ ℝ^{1 × 1}

        # np.dot(wij, δi) → (3×1)·(1×1) = (3×1)
        # g_derivada(h_j) ∈ ℝ^{1 × 3}, necesitamos trasponer el gradiente
        δj = g_derivada(h_j) * np.dot(wij, δi).T    # δj ∈ ℝ^{1 × 3}

        # ACTUALIZACIÓN DE PESOS

        # Δwij ∈ ℝ^{3 × 1} = (3×1) = (3×1)·(1×1)
        Δwij = η * np.dot(Vj.T, δi)

        # Δwjk ∈ ℝ^{3 × 3} = (3×1)·(1×3)
        Δwjk = η * np.dot(Vk.T, δj)

        wij += Δwij
        wjk += Δwjk

        # CÁLCULO DE ERROR
        error = 0.5 * np.sum((y[indice] - Vi)**2)  # escalar

        if error < error_min:
            error_min = error
            wij_min = wij.copy()
            wjk_min = wjk.copy()

        i += 1

    return wjk_min, wij_min

# =======================================================================

def predecir(x, wjk, wij):

    predicciones = []

    for ejemplo in agregar_sesgo(x):  # ejemplo ∈ ℝ^{1 × 3}
        hj = np.dot(ejemplo, wjk)     # hj ∈ ℝ^{1 × 3}
        Vj = g(hj)

        hi = np.dot(Vj, wij)          # hi ∈ ℝ^{1 × 1}
        Vi = g(hi)

        prediccion = 1 if Vi >= 0 else -1
        predicciones.append(prediccion)

    return np.array(predicciones).reshape(-1, 1)

# =============================================================================

def porcentaje_aciertos(y_real, y_predicho):

    correctos = np.sum(y_real == y_predicho)
    total = len(y_real)
    return (correctos / total) * 100


In [ ]:

# Leer el archivo
with open('numeros.txt', 'r') as f:
    lineas = [line.strip() for line in f if line.strip() != '']

# Agrupar de a 7 filas para formar cada número
num_digitos = len(lineas) // 7
X = []
y = []

for i in range(num_digitos):
    bloque = lineas[i*7:(i+1)*7]
    matriz = np.array([[int(p) for p in fila.split()] for fila in bloque])

    # if matriz.shape != (7, 5):
    #     print(f"Error en el dígito {i}: forma {matriz.shape}")

    X.append(matriz.flatten())  # convertir 7x5 a vector de 35 elementos

    # Etiquetamos según si el número es par o impar
    digito = i % 10
    etiqueta = 1 if digito % 2 == 0 else -1
    y.append([etiqueta])

X = np.array(X)
y = np.array(y)

print("Matriz de entrada X:", X.shape)
print("Vector de etiquetas y:", y.shape)


FileNotFoundError: [Errno 2] No such file or directory: 'numeros.txt'

para par o impar

In [ ]:

# Perceptrón Multicapa
def perceptron_multicapa(X, y, η=0.05, COTA=100000):
    np.random.seed(42)

    y = g(y)  # Escalar etiquetas al rango (-1, 1)

    # X ∈ ℝ^{p × 35} → +1 (sesgo) → X ∈ ℝ^{p × 36}
    X = agregar_sesgo(X)
    p, n = X.shape  # n = 36 después de agregar sesgo

    cantidad_ocultas = 1  # Solo 1 neurona en la capa oculta

    # Inicialización de pesos:
    # Wjk ∈ ℝ^{36 × 1}
    # Wij ∈ ℝ^{1 × 1}
    Wjk = np.random.uniform(-1, 1, (n, cantidad_ocultas))
    Wij = np.random.uniform(-1, 1, (cantidad_ocultas, 1))

    i = 0
    error = 50000
    error_min = 1

    while error > 0 and i < COTA:
        μ = np.random.randint(0, p)
        x_μ = X[μ].reshape(1, -1)        # x_μ ∈ ℝ^{1 × 36}
        y_μ = y[μ].reshape(1, -1)        # y_μ ∈ ℝ^{1 × 1}

        # Propagación hacia adelante
        h_j = np.dot(x_μ, Wjk)           # h_j ∈ ℝ^{1 × 1}
        v_j = g(h_j)                     # v_j ∈ ℝ^{1 × 1}

        h_i = np.dot(v_j, Wij)           # h_i ∈ ℝ^{1 × 1}
        v_i = g(h_i)                     # v_i ∈ ℝ^{1 × 1}

        # Retropropagación
        δ_i = g_derivada(h_i) * (y_μ - v_i)          # δ_i ∈ ℝ^{1 × 1}
        δ_j = g_derivada(h_j) * np.dot(δ_i, Wij.T)   # δ_j ∈ ℝ^{1 × 1}

        # Actualización de pesos
        ΔWij = η * np.dot(v_j.T, δ_i)     # ΔWij ∈ ℝ^{1 × 1}
        ΔWjk = η * np.dot(x_μ.T, δ_j)     # ΔWjk ∈ ℝ^{36 × 1}

        Wij += ΔWij
        Wjk += ΔWjk

        # Cálculo del error cuadrático medio
        error = 0.5 * np.sum((y_μ - v_i)**2)

        if error < error_min:
            error_min = error
            mejor_Wij = Wij.copy()
            mejor_Wjk = Wjk.copy()

        i += 1

    return mejor_Wjk, mejor_Wij


# Predicción
def predecir(X, Wjk, Wij):
    predicciones = []
    X = agregar_sesgo(X)
    for x_μ in X:
        x_μ = x_μ.reshape(1, -1)

        h_j = np.dot(x_μ, Wjk)
        v_j = g(h_j)

        h_i = np.dot(v_j, Wij)
        v_i = g(h_i)

        salida = 1 if v_i >= 0 else -1
        predicciones.append(salida)
    return predicciones
def comparar_resultados(X, y, Wjk, Wij):
    # Realizamos las predicciones usando el perceptrón
    predicciones = predecir(X, Wjk, Wij)

    # Comparamos los resultados esperados vs los predichos
    comparacion = [(y[μ], predicciones[μ]) for μ in range(len(y))]

    # Contamos cuántas predicciones son correctas
    correctas = sum([1 for (esperado, predicho) in comparacion if esperado == predicho])
    total = len(y)
    exactitud = correctas / total * 100  # Porcentaje de precisión

    return comparacion, exactitud

# Entrenamos el perceptrón multicapa
mejor_Wjk, mejor_Wij = perceptron_multicapa(X, y, η=0.05, COTA=100000)

# Luego puedes usar los pesos obtenidos para predecir y comparar los resultados
comparacion, exactitud = comparar_resultados(X, y, mejor_Wjk, mejor_Wij)
print(f"Exactitud del modelo: {exactitud:.2f}%")
print(comparar_resultados(X, y, mejor_Wjk, mejor_Wij))




In [ ]:


def cargar_datos_numericos(path='numeros.txt'):
    with open(path, 'r') as f:
        lineas = [line.strip() for line in f if line.strip() != '']
        #strip() es un método de las cadenas de texto que elimina los espacios en blanco
        #if line.strip() != '': Esta condición asegura que las líneas que, después de aplicar strip(), quedan vacías (es decir, solo contenían espacios o saltos de línea) sean descartadas.
        #line.strip() != '' significa que solo se incluirán en la lista aquellas líneas que no están vacías después de eliminar los espacios en blanco.
    num_digitos = len(lineas) // 7  # Cada número ocupa 7 líneas
    X = []
    y = []

    for i in range(num_digitos):
        bloque = lineas[i*7:(i+1)*7]  # Extrae las 7 líneas del dígito
        matriz = np.array([[int(p) for p in fila.split()] for fila in bloque]) #Aca con split separo cada elemento, despues los hago una lista de enteros, despues con eso creo el array
        X.append(matriz.flatten())  # Convierte 7x5 → vector de 35 elementos  //  .flatten aplana la matriz a un vector
        etiqueta = i % 10  # Asume que los dígitos están en orden de 0 a 9 -> porque en el txt estan ordenados
        y.append(etiqueta)

    return np.array(X), np.array(y)

# Entrenamiento de perceptrón multicapa
# ------------------------------------------------------------------------------------------------------------------------
def perceptron_multicapa(X, y, η=0.05, COTA=100000):
    np.random.seed(42)

    clases = 10
    y_onehot = np.zeros((y.size, clases))       # y_onehot ∈ ℝ^{p × 10}
    y_onehot[np.arange(y.size), y] = 1          # One-hot encoding  (el one hot me sirve para poder etiquetar de una forma mas comoda mis y supra mu)

    X = agregar_sesgo(X)                         # X ∈ ℝ^{p × 36} (35 + 1 de sesgo)
    p, n = X.shape

    # Pesos para cada capa: entrada → oculta1 → oculta2 → salida
    W1 = np.random.uniform(-1, 1, (n, 20))        # W1 ∈ ℝ^{36 × 20}
    W2 = np.random.uniform(-1, 1, (20, 15))       # W2 ∈ ℝ^{20 × 15}
    W3 = np.random.uniform(-1, 1, (15, 10))       # W3 ∈ ℝ^{15 × 10}
    #Aca sigo el consejo del profe de que me sirve ampliar mi capa para aumentar mi capacidad de generalizacion porque de otra forma no me funciionaba
    #(Originalmente tenia una sola capa oculta de 10 neuronas y a lo maximo que llegue fue a un 80% de acierto)

    error = 50000
    error_min = 1
    i = 0

    mejor_W1 = W1.copy()
    mejor_W2 = W2.copy()
    mejor_W3 = W3.copy()

    while error > 0 and i < COTA:
        μ = np.random.randint(0, p)                  # Selecciona un patrón aleatorio
        x_μ = X[μ].reshape(1, -1)                    # x_μ ∈ ℝ^{1 × 36}
        y_μ = y_onehot[μ].reshape(1, -1)             # y_μ ∈ ℝ^{1 × 10}

        # Forward
        h1 = np.dot(x_μ, W1)                         # h1 ∈ ℝ^{1 × 20}
        v1 = g(h1)                                   # v1 ∈ ℝ^{1 × 20}

        h2 = np.dot(v1, W2)                          # h2 ∈ ℝ^{1 × 15}
        v2 = g(h2)                                   # v2 ∈ ℝ^{1 × 15}

        h3 = np.dot(v2, W3)                          # h3 ∈ ℝ^{1 × 10}
        v3 = g(h3)                                   # v3 ∈ ℝ^{1 × 10} (salida final)

        # Backpropagation
        δ3 = g_derivada(h3) * (y_μ - v3)             # δ3 ∈ ℝ^{1 × 10}
        δ2 = g_derivada(h2) * np.dot(δ3, W3.T)       # δ2 ∈ ℝ^{1 × 15}
        δ1 = g_derivada(h1) * np.dot(δ2, W2.T)       # δ1 ∈ ℝ^{1 × 20}

        # Actualización de pesos
        W3 += η * np.dot(v2.T, δ3)                   # ΔW3 ∈ ℝ^{15 × 10}
        W2 += η * np.dot(v1.T, δ2)                   # ΔW2 ∈ ℝ^{20 × 15}
        W1 += η * np.dot(x_μ.T, δ1)                  # ΔW1 ∈ ℝ^{36 × 20}

        # Cálculo de error cuadrático
        error = 0.5 * np.sum((y_μ - v3)**2)

        if error < error_min:
            error_min = error
            mejor_W1 = W1.copy()
            mejor_W2 = W2.copy()
            mejor_W3 = W3.copy()

        i += 1

    return mejor_W1, mejor_W2, mejor_W3
#por comodidad a la hora de poder estudiarlo y entender mejor donde estaba mi fallaa no hice un bucle si no que esta cada capa recorrida "a mano"


def predecir(X, W1, W2, W3):
    X = agregar_sesgo(X)
    predicciones = []

    for x_μ in X:
        x_μ = x_μ.reshape(1, -1)

        h1 = np.dot(x_μ, W1)
        v1 = g(h1)

        h2 = np.dot(v1, W2)
        v2 = g(h2)

        h3 = np.dot(v2, W3)
        v3 = g(h3)

        predicciones.append(np.argmax(v3))

    return np.array(predicciones)
#Lo mismo aca para predecir saque los bucles porque me volvi loco tratando de encontrr donde me equivocaba

def comparar_resultados(X, y, W1, W2, W3):
    y_pred = predecir(X, W1, W2, W3)
    exactitud = np.mean(y_pred == y) * 100    #aca creo una lista con true/false y con mean cuento mi porcentaje de aciertos
    return list(zip(y, y_pred)), exactitud # zip me "tuplea los indices" de "y" e "y_pred" para comparar manualmetnte que me compile bien // lo hice para ver que efectivamente compare bien

# No es tan lindo como el de mati pero anda :)


# Cargar datos antes de entrenar
X, y = cargar_datos_numericos()

mejor_W1, mejor_W2, mejor_W3 = perceptron_multicapa(X, y, η=0.05, COTA=100000)

comparacion, exactitud = comparar_resultados(X, y, mejor_W1, mejor_W2, mejor_W3)
print(f"Exactitud del modelo: {exactitud}%")
# print(comparacion)


FileNotFoundError: [Errno 2] No such file or directory: 'numeros.txt'